# 1. Carpetas base e instalación de librerías

In [2]:
DATASET_FOLDER = '/content/drive/MyDrive/Lung_Dataset'
IMAGES_FOLDER = '/content/drive/MyDrive/Lung_Dataset/Imagenes'
IMAGES_NRRD_FOLDER = '/content/drive/MyDrive/Lung_Dataset/Imagenes_NRRD'
IMAGES_PRED_NRRD_FOLDER = '/content/drive/MyDrive/Lung_Dataset/Imagenes_Pred_NRRD'
SEGMENTATION_CODE_FOLDER = '/content/drive/MyDrive/Codigo_TFM/Lung/Codigo_segunda_entrega/Segmentacion'
MODEL_FOLDER = '/content/drive/MyDrive/Codigo_TFM/Lung/Codigo_segunda_entrega/Model'

In [3]:
from google.colab import drive
drive.mount('/content/drive')
!pip install SimpleITK
!pip install -r '/content/drive/MyDrive/Codigo_TFM/Lung/Codigo_segunda_entrega/Segmentacion/requirements.txt'
!pip install pydicom
!pip install pyplastimatch
!pip install dcm2niix
!pip install pydicom_seg
!pip install dicom2nifti
!pip install pyradiomics

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 MB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 28.1 MB/s eta 0:00:00
  Attempting uninstall: xlrd
    Found existing installation: xlrd 2.0.1
    Uninstalling xlrd-2.0.1:
      Successfully uninstalled xlrd-2.0.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 MB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.6/25.6 MB 66.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 MB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

# 2. Importación de módulos

In [14]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import gzip
import cv2
import csv
from PIL import Image

from radiomics import featureextractor
import radiomics
from radiomics import firstorder, glcm, glrlm, glszm, gldm, ngtdm, shape, shape2D
import SimpleITK as sitk

import pywt
import pydicom
from pydicom import dcmread
import pyplastimatch
import dcm2niix
import pydicom_seg
import dicom2nifti

from sklearn.model_selection import train_test_split, cross_validate, KFold, cross_val_predict, cross_val_score
from sklearn import metrics
from sklearn.decomposition import PCA

from sklearn import preprocessing

import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras import layers, models
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from keras.utils import plot_model

import random
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler

import sys
sys.path.insert(0,SEGMENTATION_CODE_FOLDER)
from TheDuneAI import ContourPilot as cp

# 3. Funciones para preprocesar los datos

## 3.1 Para los datos genómicos

In [5]:
def eliminaOutliers(df):
	# Usamos el algoritmo de envolvente elíptica para detectar los outliers. Eliminaremos un 10% de los valores anómalos
	enveliptica = EllipticEnvelope(contamination=0.10, support_fraction=0.8, random_state=42)
	df_outliers = enveliptica.fit_predict(df)
	df = df[df_outliers==1]
	return df

def eliminaAtributosCorrelacionados(df):
	matriz_corr = df.corr().abs()
	triang_superior = matriz_corr.where(np.triu(np.ones(matriz_corr.shape), k=1).astype(bool))
	columnas_correlacionadas = [column for column in triang_superior.columns if any(triang_superior[column] > 0.90)]
	df.drop(columnas_correlacionadas, axis=1, inplace=True)
	return df

def aplicaPCA(df):
	index_names = df.index.tolist()
	data_genoma = df.values
	pca = PCA(n_components=0.98)
	genoma_pca = pca.fit_transform(data_genoma)
	pca_df = pd.DataFrame(data=genoma_pca, index=index_names)

	return pca_df

In [6]:
def preprocesaDatosGenomicos(path_dataset):
	print("Preprocesando datos genómicos")
	path_genoma_txt = os.path.join(path_dataset, 'GSE103584_R01_NSCLC_RNAseq.txt.gz')
	df_genoma = pd.read_csv(gzip.open(path_genoma_txt), sep='\t', index_col=0).fillna(0)
	df_genoma = (df_genoma.T)
	#---------- PREPROCESADO DE LOS DATOS ----------#
	#---------- 1. Estimación de valores ausentes ----------#
	#Sustituimos los valores ausentes por la media
	df_genoma.fillna(df_genoma.mean(), inplace=True)
	#---------- 2. Normalización ----------#
	#Normalizamos los datos
	scaler = preprocessing.MinMaxScaler()
	pacientes_id = df_genoma.index.values
	df_genoma = pd.DataFrame(scaler.fit_transform(df_genoma),columns = df_genoma.columns, index = pacientes_id)

	#---------- 3. Reducción de dimensionalidad ----------#
	#Reducimos el número de columnas para evitar descompensar el dataset final
	display(df_genoma)
	df_genoma = aplicaPCA(df_genoma)

	#---------- 4. Tratamiento de outliers ------------#
	display(df_genoma)
	df_genoma = eliminaOutliers(df_genoma)
	display(df_genoma)
	return df_genoma

## 3.2 Para los los datos clínicos

In [7]:
def preprocesarFechas (df_pacientes):
	#Para las fechas eliminamos los labels Not Collected y las convertimos a valor numérico timestamp
	#Para Date of Recurrence
	df_pacientes["Date of Recurrence"].replace("Not Collected",np.nan, inplace=True)
	df_pacientes["Date of Recurrence"].fillna("5/5/1995", inplace=True)
	df_pacientes["Date of Recurrence"] = pd.to_datetime(df_pacientes['Date of Recurrence']).astype(int)

	#Para CT Date
	df_pacientes["CT Date"].replace("Not Collected",np.nan, inplace=True)
	df_pacientes["CT Date"].fillna("5/5/1995", inplace=True)
	df_pacientes["CT Date"] = pd.to_datetime(df_pacientes['CT Date']).astype(int)

	#Para PET Date
	df_pacientes["PET Date"].replace("Not Collected",np.nan, inplace=True)
	df_pacientes["PET Date"].fillna("5/5/1995", inplace=True)
	df_pacientes["PET Date"] = pd.to_datetime(df_pacientes['PET Date']).astype(int)

	#Para Date of Death
	df_pacientes["Date of Death"].replace("Not Collected",np.nan, inplace=True)
	df_pacientes["Date of Death"].fillna("5/5/1995", inplace=True)
	df_pacientes["Date of Death"] = pd.to_datetime(df_pacientes['Date of Death']).astype(int)

	#Para Date of Last Known Alive
	df_pacientes["Date of Last Known Alive"].replace("Not Collected",np.nan, inplace=True)
	df_pacientes["Date of Last Known Alive"].fillna("5/5/1995", inplace=True)
	df_pacientes["Date of Last Known Alive"] = pd.to_datetime(df_pacientes['Date of Last Known Alive']).astype(int)
	return df_pacientes

def preprocesaDatosClinicos(path_dataset):
	print("Preprocesando datos clínicos")
	path_csv = os.path.join(path_dataset, 'NSCLCR01Radiogenomic_DATA_LABELS_2018-05-22_1500-shifted.csv')
	df_pacientes = pd.read_csv(path_csv)
	df_pacientes.set_index('Case ID', inplace=True)
	display(df_pacientes)

	#----------PREPROCESADO DE LOS DATOS----------#
	#Los valores númericos que estan en String los convertimos a números
	df_pacientes["Weight (lbs)"] = df_pacientes["Weight (lbs)"].apply(pd.to_numeric, errors = 'coerce')
	df_pacientes["Pack Years"] = df_pacientes["Pack Years"].apply(pd.to_numeric, errors = 'coerce')

	#Preprocesamos las fechas
	df_pacientes = preprocesarFechas(df_pacientes)

	#3.Codificamos las variables categóricas en números y normalizamos las variables numéricas
	for column in df_pacientes:
		if (df_pacientes[column].dtype == np.float64) or (df_pacientes[column].dtype == np.int64):
			#Sustituimos los valores NaN (valores ausentes) por la media
			df_pacientes[column].fillna(df_pacientes[column].mean(), inplace=True)
			df_pacientes[column] = (df_pacientes[column] - df_pacientes[column].min())/(df_pacientes[column].max() - df_pacientes[column].min())
		else:
			df_pacientes = pd.get_dummies(data=df_pacientes, columns=[column], prefix=[column], prefix_sep= " | ")
	#Elimanos atributos correlacionados
	df_pacientes = eliminaAtributosCorrelacionados(df_pacientes)
	#Relacionamos las datos genómicos con los clínicos con un join
	df_pacientes.fillna(df_pacientes.mean(), inplace=True)
	display(df_pacientes)

	return df_pacientes

## 3.3. Para los datos radiómicos

### 3.3.1 Funciones para guardar las imágenes en NRRD

In [8]:
def contiene_valores(group):
    return ('CT' in group['Modality'].values) and ('SEG' in group['Modality'].values)

def obtenPathCorrecto(path):
	string_nslsc = r".\NSCLC Radiogenomics" + "\\"
	path_correcto = path.replace(string_nslsc, '').replace("\\","/")
	return path_correcto

def escribirCTNRRD(subject_id, path_nttd, path_ct_original):
    path_ct_original = os.path.join(IMAGES_FOLDER, path_ct_original)
    path_subject_nrrd = os.path.join(path_nttd, subject_id)

    if not os.path.exists(path_subject_nrrd):
        os.makedirs(path_subject_nrrd)

    path_ct_nrrd = os.path.join(path_subject_nrrd, "image.nrrd")
    if not os.path.exists(path_ct_nrrd):
        reader = sitk.ImageSeriesReader()
        dicom_names = reader.GetGDCMSeriesFileNames(path_ct_original)
        reader.SetFileNames(dicom_names)
        dicom_image = reader.Execute()
        sitk.WriteImage(dicom_image, path_ct_nrrd)

def escribirSegmentacionNRRD(subject_id, path_nttd, path_seg_original):
    path_seg_original = os.path.join(IMAGES_FOLDER, path_seg_original)
    archivo_seg = os.listdir(path_seg_original)[0]
    path_seg_original = os.path.join(path_seg_original, archivo_seg)

    path_subject_nrrd = os.path.join(path_nttd, subject_id)
    if not os.path.exists(path_subject_nrrd):
        os.makedirs(path_subject_nrrd)

    path_seg_nrrd = os.path.join(path_subject_nrrd, "mask.nrrd")
    if not os.path.exists(path_seg_nrrd):
        dcm_imagen_seg = pydicom.dcmread(path_seg_original)

        reader = pydicom_seg.MultiClassReader()
        result = reader.read(dcm_imagen_seg)

        image_data = result.data
        image = result.image
        sitk.WriteImage(image, path_seg_nrrd, True)

def guardarImagenesConSegmentacionNRRD(path_dataset):
	path_imagenes = os.path.join(path_dataset, 'Imagenes')

	df_metadata = pd.read_csv(os.path.join(path_dataset, 'metadata.csv'))
	df_metadata = df_metadata.reset_index()

	for subject_id in df_metadata['Subject ID'].unique():
		df_filtered_by_subject = df_metadata[df_metadata['Subject ID'] == subject_id]
		df_grouped_by_subject = df_filtered_by_subject.groupby('Subject ID')

		filtered = df_grouped_by_subject.filter(contiene_valores)
		if not filtered.empty:
			print("------ " + subject_id + " ------")
			#Filtramos el dataframe para solo obtener las filas que sean segmentaciones
			row_where_seg = df_filtered_by_subject[df_filtered_by_subject['Modality'] == 'SEG']
            #Obtenemos el valor del path a esa segmentacion
			path_mask = row_where_seg['File Location'].values[0]
			path_carpeta_general = path_mask.rsplit('\\', 1)[0]

            #Obtenemos el path a la imagen ct de la segmentación
			rows_where_ct = df_filtered_by_subject[df_filtered_by_subject['Modality'] == 'CT']
			row_where_ct = rows_where_ct[rows_where_ct['File Location'].str.startswith(path_carpeta_general)]
			path_ct = row_where_ct['File Location'].values[0]

			escribirCTNRRD(subject_id, IMAGES_NRRD_FOLDER, obtenPathCorrecto(path_ct))
			escribirSegmentacionNRRD(subject_id, IMAGES_NRRD_FOLDER, obtenPathCorrecto(path_mask))

def guardarImagenesSinSegmentacionNRRD(path_dataset):
	path_imagenes = os.path.join(path_dataset, 'Imagenes')

	df_metadata = pd.read_csv(os.path.join(path_dataset, 'metadata.csv'))
	df_metadata = df_metadata.reset_index()

	for subject_id in df_metadata['Subject ID'].unique():
		df_filtered_by_subject = df_metadata[df_metadata['Subject ID'] == subject_id]
		df_grouped_by_subject = df_filtered_by_subject.groupby('Subject ID')

		filtered = df_grouped_by_subject.filter(contiene_valores)
		if filtered.empty:
			print("NOT SEG ------ " + subject_id + " ------")

			rows_where_ct = df_filtered_by_subject[df_filtered_by_subject['Modality'] == 'CT']
			rows_with_no_pet = rows_where_ct[~rows_where_ct['File Location'].str.contains('PET')]
			max_num_files = rows_with_no_pet['Number of Images'].max()
			row_max_num_files = rows_with_no_pet[rows_with_no_pet['Number of Images'] == max_num_files]
			num_rows = row_max_num_files.shape
			num_rows, num_columns = row_max_num_files.shape
			if num_rows > 0:
				path_ct = row_max_num_files['File Location'].values[0]
				escribirCTNRRD(subject_id, IMAGES_PRED_NRRD_FOLDER, obtenPathCorrecto(path_ct))


### 3.3.2 Funciones para predecir las segmentaciones restantes

In [9]:
def predecirSegmentacionNRRD(path_dataset):
	path_imagenes = os.path.join(path_dataset, 'Imagenes')
	path_modelo = os.path.join(SEGMENTATION_CODE_FOLDER, 'Modelo')
	model = cp(path_modelo,IMAGES_PRED_NRRD_FOLDER,IMAGES_PRED_NRRD_FOLDER,verbosity=True)
	model.segment()

def guardarImagenesYPredecirSegmentacionNRRD(path_dataset):
	guardarImagenesSinSegmentacionNRRD(path_dataset)
	guardarImagenesConSegmentacionNRRD(path_dataset)
	predecirSegmentacionNRRD(path_dataset)

### 3.3.3 Funciones para extrar y guardar los datos radiómicos

In [10]:
def extraerDatosRadiomicosImagen(extractor, path_nrrd):
    path_nrrd_ct = os.path.join(path_nrrd, 'image.nrrd')
    path_nrrd_seg = os.path.join(path_nrrd, 'mask.nrrd')

    image_nrrd_ct = sitk.ReadImage(path_nrrd_ct)
    image_nrrd_seg = sitk.ReadImage(path_nrrd_seg)
    try:
        firstOrderFeatures = firstorder.RadiomicsFirstOrder(image_nrrd_ct, image_nrrd_seg)
        glcmFeatures = glcm.RadiomicsGLCM(image_nrrd_ct, image_nrrd_seg)
        glrlmFeatures = glrlm.RadiomicsGLRLM(image_nrrd_ct, image_nrrd_seg)
        ngtdmFeatures = ngtdm.RadiomicsNGTDM(image_nrrd_ct, image_nrrd_seg)
        gldmFeatures = gldm.RadiomicsGLDM(image_nrrd_ct, image_nrrd_seg)
    except Exception as e:
        return np.empty(0)
    else:
        firstOrderFeatures = np.array([v for _, v in firstOrderFeatures.execute().items()])
        glcmFeatures = np.array([v for _, v in glcmFeatures.execute().items()])
        glrlmFeatures = np.array([v for _, v in glrlmFeatures.execute().items()])
        ngtdmFeatures = np.array([v for _, v in ngtdmFeatures.execute().items()])
        gldmFeatures = np.array([v for _, v in gldmFeatures.execute().items()])
        caract_radiomics = np.hstack((firstOrderFeatures, glcmFeatures, glrlmFeatures, ngtdmFeatures, gldmFeatures))
        return caract_radiomics

def extraerDatosRadiomicosCarpeta(path_carpeta, path_caract_radiomicas_parciales):
    lista_caract_radiomicas = {}
    lista_sujetos = {}
    if os.path.exists(path_caract_radiomicas_parciales):
        df_radiomicas_parciales = pd.read_csv(path_caract_radiomicas_parciales, index_col=0, header=0)
        for index, row in df_radiomicas_parciales.iterrows():
            lista_caract_radiomicas[index] = row.values
        lista_sujetos = df_radiomicas_parciales.index.tolist()

    extractor = featureextractor.RadiomicsFeatureExtractor()

    for nombre_subcarpeta in os.listdir(path_carpeta):
        if nombre_subcarpeta not in lista_sujetos:
            print("   paciente " + nombre_subcarpeta)
            path_subcarpeta= os.path.join(path_carpeta, nombre_subcarpeta)
            datos_radiomicos_imagen = extraerDatosRadiomicosImagen(extractor, path_subcarpeta)

            if datos_radiomicos_imagen.size != 0:
                lista_caract_radiomicas[nombre_subcarpeta] = datos_radiomicos_imagen
                df_radiomicas_parciales = pd.DataFrame.from_dict(lista_caract_radiomicas, orient='index')
                df_radiomicas_parciales.fillna(0, inplace=True)
                df_radiomicas_parciales.index.name='Case ID'
                df_radiomicas_parciales.to_csv(path_caract_radiomicas_parciales, index=True, header=True)
    return lista_caract_radiomicas

def procesaDatosRadiomicos(lista_caract_radiomicas):
    df_imagenes = pd.DataFrame.from_dict(lista_caract_radiomicas, orient='index')
    df_imagenes.fillna(0, inplace=True)
    scaler = preprocessing.MinMaxScaler()
    pacientes_id = df_imagenes.index.values
    caract_imagenes = pd.DataFrame(scaler.fit_transform(df_imagenes),columns = df_imagenes.columns, index = pacientes_id)
    caract_imagenes.index.name='Case ID'
    return caract_imagenes

def extraerDatosRadiomicos(path_dataset):
    print("Extrayendo datos radiómicos...")

    path_caract_radiomicas_parciales = os.path.join(path_dataset, 'Caracteristicas_extraidas/datos_radiomicos_parciales.csv')

    lista_caract_radiomicas_seg_orig = extraerDatosRadiomicosCarpeta(IMAGES_NRRD_FOLDER, path_caract_radiomicas_parciales)
    lista_caract_radiomicas = extraerDatosRadiomicosCarpeta(IMAGES_PRED_NRRD_FOLDER, path_caract_radiomicas_parciales)

    return procesaDatosRadiomicos(lista_caract_radiomicas)


# 4. Extracción de las características

## 4.1 Guardar características

In [11]:
def preprocesaYGuardaCaracteristicas(path_dataset):
    #Si las características de los datos clínicos no habían sido extraidas y guardadas, las guardamos
    path_caract_clinicas = os.path.join(path_dataset, 'Caracteristicas_extraidas/datos_clinicos.csv')
    if not os.path.isfile(path_caract_clinicas):
        caract_clinicas = preprocesaDatosClinicos(path_dataset)
        caract_clinicas.to_csv(path_caract_clinicas, index=True, header=True)

    #Si las características de los datos genómicos no habían sido extraidas y guardadas, las guardamos
    path_caract_genomicas = os.path.join(path_dataset, 'Caracteristicas_extraidas/datos_genomicos.csv')
    if not os.path.isfile(path_caract_genomicas):
        caract_genomicas = preprocesaDatosGenomicos(path_dataset)
        caract_genomicas.to_csv(path_caract_genomicas, index=True, header=True)

    #Si las características de los datos radiómicos no habían sido extraidas y guardadas, las guardamos
    path_caract_radiomicas = os.path.join(path_dataset, 'Caracteristicas_extraidas/datos_radiomicos.csv')
    if not os.path.isfile(path_caract_radiomicas):
        caract_radiomicas = extraerDatosRadiomicos(path_dataset)
        caract_radiomicas.to_csv(path_caract_radiomicas, index=True, header=True)

## 4.2 Extraer características

In [12]:
def extraeCaracteristicasCompletas(path_caract_1, path_caract_2, path_caract_3, tipo_union):
    caract_1 = pd.read_csv(path_caract_1, index_col=0, header=0)
    caract_2 = pd.read_csv(path_caract_2, index_col=0, header=0)
    caract_3 = pd.read_csv(path_caract_3, index_col=0, header=0)

    #Concatenamos las características
    if tipo_union == "outer":
        caract_merged = pd.merge(caract_1, caract_2, left_index=True, right_index=True, how="outer")
        caract = pd.merge(caract_merged, caract_3, left_index=True, right_index=True, how="outer")
    elif tipo_union == "inner":
        caract_merged = pd.merge(caract_1, caract_2, left_index=True, right_index=True, how="inner")
        caract = pd.merge(caract_merged, caract_3, left_index=True, right_index=True, how="inner")
    caract.fillna(caract.mean(), inplace=True)
    return caract

def extraeDosCaracteristicas(es_clinica, path_caract_1, path_caract_2, tipo_union):
    caract_1 = pd.read_csv(path_caract_1, index_col=0, header=0)
    caract_2 = pd.read_csv(path_caract_2, index_col=0, header=0)

    #Concatenamos las características
    if tipo_union == "outer":
        caract = pd.merge(caract_1, caract_2, left_index=True, right_index=True, how="outer")
    elif tipo_union == "inner":
        caract = pd.merge(caract_1, caract_2, left_index=True, right_index=True, how="inner")
    caract.fillna(caract.mean(), inplace=True)
    if not es_clinica:
        path_caract_clinicas = path_caract_clinicas = os.path.join(DATASET_FOLDER, 'Caracteristicas_extraidas/datos_clinicos.csv')
        caract_clinicas = pd.read_csv(path_caract_clinicas, index_col=0, header=0)
        survival_status_col = caract_clinicas['Survival Status | Alive']
        df_survival = survival_status_col.to_frame(name='Survival Status | Alive')
        caract = pd.merge(caract, df_survival, left_index=True, right_index=True, how="inner")
    return caract

def extraeUnaCaracteristica(es_clinica, path_caract, tipo_union):
    caract = pd.read_csv(path_caract, index_col=0, header=0)
    if not es_clinica:
        path_caract_clinicas = os.path.join(DATASET_FOLDER, 'Caracteristicas_extraidas/datos_clinicos.csv')
        caract_clinicas = pd.read_csv(path_caract_clinicas, index_col=0, header=0)
        survival_status_col = caract_clinicas['Survival Status | Alive']
        df_survival = survival_status_col.to_frame(name='Survival Status | Alive')
        caract = pd.merge(caract, df_survival, left_index=True, right_index=True, how="inner")
    return caract

def extraeCaractSegunInput(clinicos, genomicos, radiomicos, tipo_union):
    path_caract_clinicas = os.path.join(DATASET_FOLDER, 'Caracteristicas_extraidas/datos_clinicos.csv')
    path_caract_genomicas = os.path.join(DATASET_FOLDER, 'Caracteristicas_extraidas/datos_genomicos.csv')
    path_caract_radiomicas = os.path.join(DATASET_FOLDER, 'Caracteristicas_extraidas/datos_radiomicos.csv')
    df_caract = {}
    if clinicos and genomicos and radiomicos:
        df_caract = extraeCaracteristicasCompletas(path_caract_clinicas, path_caract_genomicas, path_caract_radiomicas, tipo_union)
    elif clinicos and genomicos and not radiomicos:
        df_caract = extraeDosCaracteristicas(clinicos, path_caract_clinicas, path_caract_genomicas, tipo_union)
    elif clinicos and not genomicos and radiomicos:
        df_caract = extraeDosCaracteristicas(clinicos, path_caract_clinicas, path_caract_radiomicas, tipo_union)
    elif not clinicos and genomicos and radiomicos:
        df_caract = extraeDosCaracteristicas(clinicos, path_caract_genomicas, path_caract_radiomicas, tipo_union)
    elif clinicos and not genomicos and not radiomicos:
        df_caract = extraeUnaCaracteristica(clinicos, path_caract_clinicas, tipo_union)
    elif not clinicos and genomicos and not radiomicos:
        df_caract = extraeUnaCaracteristica(clinicos, path_caract_genomicas, tipo_union)
    elif not clinicos and not genomicos and radiomicos:
        df_caract = extraeUnaCaracteristica(clinicos, path_caract_radiomicas, tipo_union)
    return df_caract

extraeCaractSegunInput(clinicos=False, genomicos=True, radiomicos=True, tipo_union="outer")

,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7_x,8_x,9_x,...,68_y,69_y,70_y,71_y,72_y,73_y,74_y,75_y,76_y,Survival Status | Alive
AMC-001,0.006026,-0.517859,0.053041,-0.029676,-0.051478,-0.169131,-0.009243,0.044800,-0.167539,-0.050045,...,64.320879,1749.228961,10.028697,18475.412687,0.041298,0.003921,0.374366,619.156225,0.001360,0
AMC-002,0.006026,-0.517859,0.053041,-0.029676,-0.051478,-0.169131,-0.009243,0.044800,-0.167539,-0.050045,...,51.604286,1501.787473,44.780683,77968.934357,0.031552,0.001218,0.179935,219.652110,0.000353,1
AMC-003,0.006026,-0.517859,0.053041,-0.029676,-0.051478,-0.169131,-0.009243,0.044800,-0.167539,-0.050045,...,43.422387,1543.026677,9.427991,15069.390102,0.007481,0.001153,0.353348,522.331503,0.000607,1
AMC-004,0.006026,-0.517859,0.053041,-0.029676,-0.051478,-0.169131,-0.009243,0.044800,-0.167539,-0.050045,...,77.796909,809.017034,6.160321,5852.996994,0.021878,0.005051,0.456227,322.002381,0.003008,1
AMC-005,0.006026,-0.517859,0.053041,-0.029676,-0.051478,-0.169131,-0.009243,0.044800,-0.167539,-0.050045,...,63.089273,1432.009101,24.755580,42053.203900,0.018119,0.001701,0.275254,291.661242,0.000694,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
R01-156,7.025757,-5.931159,-3.100637,9.199866,0.013380,0.004615,5.751905,-1.450845,-4.877836,-2.285581,...,118.870972,1103.112010,19.171870,26856.320566,0.241052,0.011503,0.379061,353.711469,0.004620,0
R01-157,4.181196,-5.879165,-1.256348,6.707151,-0.162189,5.030562,4.055994,1.790728,-1.541707,-0.804697,...,12.546504,699.584094,24.575979,18504.161899,0.033553,0.001914,0.222646,130.856728,0.000803,1
R01-158,14.903090,-7.677206,0.473159,8.042454,-0.167521,0.324751,6.534695,-2.529601,-5.471935,-3.321318,...,118.870972,1103.112010,19.171870,26856.320566,0.241052,0.011503,0.379061,353.711469,0.004620,1
R01-159,-8.692994,-1.580766,-3.731303,2.604348,8.956873,-0.208848,-3.691910,-2.855215,-3.791294,2.129137,...,118.870972,1103.112010,19.171870,26856.320566,0.241052,0.011503,0.379061,353.711469,0.004620,1


# 5. Implementación modelo

Probando con modelo propio

In [17]:
def creaModeloMultimodalComplejo(x_train):
    model = Sequential()
    model.add(Dense(512,input_dim = x_train.shape[1],activation="relu"))
    model.add(Dense(512,activation="relu"))
    model.add(Dense(256,activation="relu"))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1,activation = "sigmoid"))

    return model

def creaModeloMultimodalSimple(x_train):
    model = Sequential()
    model.add(Dense(64,input_dim = x_train.shape[1],activation="relu"))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))

    return model

def mostrarCurvaAprendizaje(H, num_epocas):
    plt.style.use("ggplot")
    plt.figure()
    plt.plot(np.arange(5, num_epocas), H.history["loss"][4:79], label="train_loss")
    plt.plot(np.arange(5, num_epocas), H.history["val_loss"][4:79], label="val_loss")
    plt.plot(np.arange(5, num_epocas), H.history["accuracy"][4:79], label="train_acc")
    plt.plot(np.arange(5, num_epocas), H.history["val_accuracy"][4:79], label="val_acc")
    plt.title("Training Loss and Accuracy")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss/Accuracy")
    plt.legend()
    plt.show()

def prediceSupervivenciaParaModelo(modelo, x_train, x_test, y_train):
    num_epocas = 80
    modelo.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

    H = modelo.fit(x_train, y_train, epochs=num_epocas, batch_size=16, validation_split=0.2, verbose=False)
    predictions = modelo.predict(x_test)
    #mostrarHistograma(H, num_epocas)

    return predictions

def obtenMetricas(y_test, y_pred):
    binary_predictions = (y_pred > 0.5).astype(int)

    accuracy = accuracy_score(y_test, binary_predictions)
    precision = precision_score(y_test, binary_predictions)
    recall = recall_score(y_test, binary_predictions)
    f1 = f1_score(y_test, binary_predictions)
    print('Métricas: {')
    print('     accuracy:  %.4f' % accuracy)
    print('     precision: %.4f' % precision)
    print('     recall: %.4f' % recall)
    print('     f1:   %.4f' % f1)
    print('}')

def cargaCaracteristicasYPrediceSegunInput(clinicos, genomicos, radiomicos, tipo_union, texto_datos, texto_union):
    df_caract = extraeCaractSegunInput(clinicos, genomicos, radiomicos, tipo_union)
    df_caract = df_caract._get_numeric_data()

    x = df_caract.values.astype(int)
    y = df_caract['Survival Status | Alive'].values.astype(int)
    #print("Y shape: ", y.shape)

    predecirSupervivencia(x, y, x.shape, texto_datos, texto_union)

def printTextos(texto_datos, texto_union):
    print(texto_datos)
    print(texto_union)

def predecirSupervivencia(x, y, x_shape, texto_datos, texto_union):
	x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.15)
	x_train_val, x_test_val, y_train_val, y_test_val = train_test_split(x_train, y_train, test_size = 0.15)
	#print("X_train shape:", x_train.shape)
	#print("y_train shape:", y_train.shape)

	conjunto_modelos = {}
	conjunto_modelos["Red neuronal propia"] = creaModeloMultimodalComplejo(x_train)
	plot_model(conjunto_modelos["Red neuronal propia"], show_shapes=True, to_file='modelo_complejo_estructura.png')

	conjunto_modelos["Clasificador MLP"] = MLPClassifier(solver='adam', random_state=42)
	conjunto_modelos["Regresión logística"] = LogisticRegression(random_state=42)
	conjunto_modelos["K vecinos"] = KNeighborsClassifier(n_neighbors=6,  metric="euclidean")

	for modelo in conjunto_modelos:

		print("-----------------------")
		printTextos(texto_datos, texto_union)
		print("Tamaño conjunto de datos: ", x_shape)
		print("Modelo:  " + modelo)
		if modelo == "Red neuronal propia":
			y_pred = prediceSupervivenciaParaModelo(conjunto_modelos[modelo], x_train, x_test, y_train)
			obtenMetricas(y_test, y_pred)
		elif modelo == "K vecinos":
			print('Métricas: {')
			precision = cross_val_score(conjunto_modelos[modelo], x_train, y_train, cv=10, scoring='precision')
			print('     precision:  %.4f' % precision.mean())
			accuracy = cross_val_score(conjunto_modelos[modelo], x_train, y_train, cv=10, scoring='accuracy')
			print('     accuracy:  %.4f' % accuracy.mean())
			recall = cross_val_score(conjunto_modelos[modelo], x_train, y_train, cv=10, scoring='recall')
			print('     recall:  %.4f' % recall.mean())
			f1 = cross_val_score(conjunto_modelos[modelo], x_train, y_train, cv=10, scoring='f1')
			print('     f1:  %.4f' % f1.mean())
			print('}')
		else:
			conjunto_modelos[modelo] = conjunto_modelos[modelo].fit(x_train, y_train)
			y_pred = conjunto_modelos[modelo].predict(x_test)
			obtenMetricas(y_test, y_pred)
		print("-----------------------\n")

def muestraTodasLasPrediccionesPosibles():
    tipos_union = {"outer", "inner"}
    #Solo datos clínicos
    texto_union = "Tipo de unión de datos: ninguna"
    texto_datos = "Tipos de datos: datos clínicos"
    cargaCaracteristicasYPrediceSegunInput(clinicos=True, genomicos=False, radiomicos=False, tipo_union=None, texto_datos=texto_datos, texto_union=texto_union)

    #Solo datos genómicos
    texto_union = "Tipo de unión de datos: ninguna"
    texto_datos = "Tipos de datos usados: datos genómicos"
    cargaCaracteristicasYPrediceSegunInput(clinicos=False, genomicos=True, radiomicos=False, tipo_union=None, texto_datos=texto_datos, texto_union=texto_union)

    #Solo datos radiómicos
    texto_union = "Tipo de unión de datos: ninguna"
    texto_datos = "Tipos de datos usados: datos radiómicos"
    cargaCaracteristicasYPrediceSegunInput(clinicos=False, genomicos=False, radiomicos=True, tipo_union=None, texto_datos=texto_datos, texto_union=texto_union)

    for union in tipos_union:
        texto_union = "Tipo de unión de datos: " + union
        #Datos clínicos y genómicos
        texto_datos = "Tipos de datos usados: clínicos y genómicos"
        cargaCaracteristicasYPrediceSegunInput(clinicos=True, genomicos=True, radiomicos=False, tipo_union=union, texto_datos=texto_datos, texto_union=texto_union)

        #Datos clínicos y radiómicos
        texto_union = "Tipo de unión de datos: " + union
        texto_datos = "Tipos de datos usados: clínicos y radiómicos"
        cargaCaracteristicasYPrediceSegunInput(clinicos=True, genomicos=False, radiomicos=True, tipo_union=union, texto_datos=texto_datos, texto_union=texto_union)

        #Datos genómicos y radiómicos
        texto_union = "Tipo de unión de datos: " + union
        texto_datos = "Tipos de datos usados: genómicos y radiómicos"
        cargaCaracteristicasYPrediceSegunInput(clinicos=False, genomicos=True, radiomicos=True, tipo_union=union, texto_datos=texto_datos, texto_union=texto_union)

        #Todos los datos: clínicos, genómicos y radiómicos
        texto_union = "Tipo de unión de datos: " + union
        texto_datos = "Tipos de datos usados: clínicos, genómicos y radiómicos"
        cargaCaracteristicasYPrediceSegunInput(clinicos=True, genomicos=True, radiomicos=True, tipo_union=union, texto_datos=texto_datos, texto_union=texto_union)

muestraTodasLasPrediccionesPosibles()

-----------------------
Tipos de datos: datos clínicos
Tipo de unión de datos: ninguna
Tamaño conjunto de datos:  (211, 79)
Modelo:  Red neuronal propia
1/1 [==============================] - 0s 89ms/step
Métricas: {
     accuracy:  1.0000
     precision: 1.0000
     recall: 1.0000
     f1:   1.0000
}
-----------------------

-----------------------
Tipos de datos: datos clínicos
Tipo de unión de datos: ninguna
Tamaño conjunto de datos:  (211, 79)
Modelo:  Clasificador MLP


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Métricas: {
     accuracy:  0.7500
     precision: 0.7619
     recall: 0.8421
     f1:   0.8000
}
-----------------------

-----------------------
Tipos de datos: datos clínicos
Tipo de unión de datos: ninguna
Tamaño conjunto de datos:  (211, 79)
Modelo:  Regresión logística
Métricas: {
     accuracy:  1.0000
     precision: 1.0000
     recall: 1.0000
     f1:   1.0000
}
-----------------------

-----------------------
Tipos de datos: datos clínicos
Tipo de unión de datos: ninguna
Tamaño conjunto de datos:  (211, 79)
Modelo:  K vecinos
Métricas: {
     precision:  0.8560
     accuracy:  0.8657
     recall:  0.9846
     f1:  0.9146
}
-----------------------

-----------------------
Tipos de datos usados: datos genómicos
Tipo de unión de datos: ninguna
Tamaño conjunto de datos:  (110, 121)
Modelo:  Red neuronal propia
1/1 [==============================] - 0s 63ms/step
Métricas: {
     accuracy:  0.5294
     precision: 0.5000
     recall: 1.0000
     f1:   0.6667
}
----------------------

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


     f1:  0.6715
}
-----------------------

-----------------------
Tipos de datos usados: clínicos y genómicos
Tipo de unión de datos: inner
Tamaño conjunto de datos:  (110, 199)
Modelo:  Red neuronal propia
1/1 [==============================] - 0s 64ms/step
Métricas: {
     accuracy:  0.7059
     precision: 0.6667
     recall: 1.0000
     f1:   0.8000
}
-----------------------

-----------------------
Tipos de datos usados: clínicos y genómicos
Tipo de unión de datos: inner
Tamaño conjunto de datos:  (110, 199)
Modelo:  Clasificador MLP
Métricas: {
     accuracy:  0.5882
     precision: 0.6154
     recall: 0.8000
     f1:   0.6957
}
-----------------------

-----------------------
Tipos de datos usados: clínicos y genómicos
Tipo de unión de datos: inner
Tamaño conjunto de datos:  (110, 199)
Modelo:  Regresión logística
Métricas: {
     accuracy:  0.6471
     precision: 0.6429
     recall: 0.9000
     f1:   0.7500
}
-----------------------

-----------------------
Tipos de datos usad

...

     f1:  0.5605
}
-----------------------

-----------------------
Tipos de datos usados: clínicos y genómicos
Tipo de unión de datos: outer
Tamaño conjunto de datos:  (211, 199)
Modelo:  Red neuronal propia
1/1 [==============================] - 0s 60ms/step
Métricas: {
     accuracy:  0.7812
     precision: 0.8182
     recall: 0.8571
     f1:   0.8372
}
-----------------------

-----------------------
Tipos de datos usados: clínicos y genómicos
Tipo de unión de datos: outer
Tamaño conjunto de datos:  (211, 199)
Modelo:  Clasificador MLP


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Métricas: {
     accuracy:  0.6875
     precision: 0.7619
     recall: 0.7619
     f1:   0.7619
}
-----------------------

-----------------------
Tipos de datos usados: clínicos y genómicos
Tipo de unión de datos: outer
Tamaño conjunto de datos:  (211, 199)
Modelo:  Regresión logística
Métricas: {
     accuracy:  0.8750
     precision: 0.9048
     recall: 0.9048
     f1:   0.9048
}
-----------------------

-----------------------
Tipos de datos usados: clínicos y genómicos
Tipo de unión de datos: outer
Tamaño conjunto de datos:  (211, 199)
Modelo:  K vecinos
Métricas: {
     precision:  0.8362
     accuracy:  0.8039
     recall:  0.9051
     f1:  0.8680
}
-----------------------

-----------------------
Tipos de datos usados: clínicos y radiómicos
Tipo de unión de datos: outer
Tamaño conjunto de datos:  (211, 156)
Modelo:  Red neuronal propia
1/1 [==============================] - 0s 59ms/step
Métricas: {
     accuracy:  0.8125
     precision: 0.8125
     recall: 1.0000
     f1:   0.8

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


     recall:  0.6527
     f1:  0.6194
}
-----------------------

-----------------------
Tipos de datos usados: clínicos, genómicos y radiómicos
Tipo de unión de datos: outer
Tamaño conjunto de datos:  (211, 276)
Modelo:  Red neuronal propia
1/1 [==============================] - 0s 60ms/step
Métricas: {
     accuracy:  0.5938
     precision: 0.6129
     recall: 0.9500
     f1:   0.7451
}
-----------------------

-----------------------
Tipos de datos usados: clínicos, genómicos y radiómicos
Tipo de unión de datos: outer
Tamaño conjunto de datos:  (211, 276)
Modelo:  Clasificador MLP
Métricas: {
     accuracy:  0.3750
     precision: 0.5000
     recall: 0.1000
     f1:   0.1667
}
-----------------------

-----------------------
Tipos de datos usados: clínicos, genómicos y radiómicos
Tipo de unión de datos: outer
Tamaño conjunto de datos:  (211, 276)
Modelo:  Regresión logística
Métricas: {
     accuracy:  0.6250
     precision: 0.6250
     recall: 1.0000
     f1:   0.7692
}
-----------